In [1]:
from pyspark.sql import SparkSession

from pyspark.sql.functions import *
from pyspark.sql.types import *

spark = SparkSession \
    .builder \
    .appName("limpieza") \
    .getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


2024-11-19T23:41:14,920 WARN [Thread-4] org.apache.hadoop.util.NativeCodeLoader - Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


# Receptivo
Contiene los datos de ingreso al país

In [2]:
df_receptivo = spark.read.csv('../../obligatorio/lnd/receptivo', header=True, sep=',',inferSchema=True)

Asi podemos visualizar los tipos de datos y columnas

In [3]:
df_receptivo.printSchema()

root
 |-- IdIngresos: integer (nullable = true)
 |-- Lugar Ingreso: string (nullable = true)
 |-- IdTranspIngreso: integer (nullable = true)
 |-- Transporte Internacional de Ingreso: string (nullable = true)
 |-- FechaIngreso: date (nullable = true)
 |-- IdFecIng: integer (nullable = true)
 |-- FechaEgreso: date (nullable = true)
 |-- IdFecEgr: integer (nullable = true)
 |-- IdNacionalidad: integer (nullable = true)
 |-- Pais: string (nullable = true)
 |-- IdResidencia: integer (nullable = true)
 |-- Residencia: string (nullable = true)
 |-- IdMotivo: integer (nullable = true)
 |-- Motivo: string (nullable = true)
 |-- IdOcupacion: integer (nullable = true)
 |-- Ocupacion: string (nullable = true)
 |-- IsEstudio: integer (nullable = true)
 |-- Estudio: string (nullable = true)
 |-- IdDestinoLocalidad: integer (nullable = true)
 |-- Localidad: string (nullable = true)
 |-- IdDepartamentoDestino: integer (nullable = true)
 |-- Departamento: string (nullable = true)
 |-- IdOtroDepartament

### Descripción de las Columnas

- Lugar Ingreso es por donde ingresó y tiene su propio ID (IdIngresos).
- Transporte Internacional de Ingreso es el medio por el cual ingresó y tiene su propio ID (IdTranspIngreso).
- Fecha Ingreso corresponde a la fecha de ingreso con su ID (IdFecIng).
- Fecha Egreso corresponde a la fecha de salida con su ID (IdFecEgr).
- Pais es la nacionalidad de quien ingresó y tiene su propio ID (IdNacionalidad).
- Residencia (lugar de residencia del visitante) con su ID (IdResidencia).
- Motivo de ingreso y cuenta con su propio ID (IdMotivo).
- Ocupacion corresponde a la profesión del turista y tiene un ID (IdOcupacion).
- Estudio es el nivel de estudios del visitante, que tiene su propio ID (IsEstudio), mas adelante le vamos a corregir el nombre de ID para que quede correcto (IdEstudio).
- Destino Localidad es el principal destino de la visita con su ID (IdDestinoLocalidad).
- Departamento corresponde al departamento de destino principal con su ID (IdDepartamentoDestino).
- Otro Departamento es un posible segundo destino y tiene su ID (IdOtroDepartamento).
- Otra Localidad corresponde a una segunda localidad visitada con su ID (IdOtraLocalidad).
- Alojamiento es el tipo de alojamiento utilizado con su ID (IdAlojamiento).
- Transporte Local corresponde al transporte utilizado durante la estadía y tiene un ID (IdTranspLocal).
- Lugar Egreso es por donde se egresó y cuenta con su ID (IdEgresos).
- Transporte Internacional de Egreso corresponde al transporte utilizado para salir del país con su ID (IdTranspEgreso).
- Destino es la zona de destino principal con su ID (IdDestino).
- Estadia representa los días que permaneció en el país.
- Gente es la cantidad de personas del grupo encuestado.
- Gasto Total, Gasto Alojamiento, Gasto Alimentacion, Gasto Transporte, Gasto Cultural, Gasto Tours, Gasto Compras y Gasto Otros corresponde a todos los tipos de gastos posibles, expresado en dólares americanos.
- Coef es el coeficiente de expansión del gasto.
- CoefTot es el coeficiente de expansión de las personas.

In [4]:
# Quitar columnas que no usamos

df_receptivo = df_receptivo.drop(
    "IdIngresos", 
    "IdTranspIngreso", 
    "IdFecIng", 
    "IdFecEgr", 
    "IdNacionalidad", 
    "IdResidencia", 
    "IdMotivo", 
    "IdOcupacion", 
    "IsEstudio", 
    "IdDestinoLocalidad", 
    "IdDepartamentoDestino",
    "IdOtroDepartamento",
    "IdOtraLocalidad",
    "IdAlojamiento", 
    "IdTranspLocal", 
    "IdEgresos",
    "IdTranspEgreso",
    "IdDestino"
);


df_receptivo.printSchema()

root
 |-- Lugar Ingreso: string (nullable = true)
 |-- Transporte Internacional de Ingreso: string (nullable = true)
 |-- FechaIngreso: date (nullable = true)
 |-- FechaEgreso: date (nullable = true)
 |-- Pais: string (nullable = true)
 |-- Residencia: string (nullable = true)
 |-- Motivo: string (nullable = true)
 |-- Ocupacion: string (nullable = true)
 |-- Estudio: string (nullable = true)
 |-- Localidad: string (nullable = true)
 |-- Departamento: string (nullable = true)
 |-- Otro Departamento: string (nullable = true)
 |-- Otra Localidad: string (nullable = true)
 |-- Alojamiento: string (nullable = true)
 |-- TransporteLocal: string (nullable = true)
 |-- Lugar Egreso: string (nullable = true)
 |-- Transporte Internacional de Egreso: string (nullable = true)
 |-- Destino: string (nullable = true)
 |-- Estadia: integer (nullable = true)
 |-- Gente: integer (nullable = true)
 |-- GastoTotal: double (nullable = true)
 |-- GastoAlojamiento: double (nullable = true)
 |-- GastoAliment

In [5]:
from pyspark.sql import functions as F

# Reemplazar tildes en la columna "Lugar Ingreso"
df_receptivo = df_receptivo.withColumn(
    "Lugar Ingreso",
    F.regexp_replace("Lugar Ingreso", "Á", "A")
)

df_receptivo = df_receptivo.withColumn(
    "Lugar Ingreso",
    F.regexp_replace("Lugar Ingreso", "É", "E")
)

df_receptivo = df_receptivo.withColumn(
    "Lugar Ingreso",
    F.regexp_replace("Lugar Ingreso", "Í", "I")
)

df_receptivo = df_receptivo.withColumn(
    "Lugar Ingreso",
    F.regexp_replace("Lugar Ingreso", "Ó", "O")
)

df_receptivo = df_receptivo.withColumn(
    "Lugar Ingreso",
    F.regexp_replace("Lugar Ingreso", "Ú", "U")
)

df_receptivo = df_receptivo.withColumn(
    "Lugar Ingreso",
    F.regexp_replace("Lugar Ingreso", "á", "a")
)

df_receptivo = df_receptivo.withColumn(
    "Lugar Ingreso",
    F.regexp_replace("Lugar Ingreso", "é", "e")
)

df_receptivo = df_receptivo.withColumn(
    "Lugar Ingreso",
    F.regexp_replace("Lugar Ingreso", "í", "i")
)

df_receptivo = df_receptivo.withColumn(
    "Lugar Ingreso",
    F.regexp_replace("Lugar Ingreso", "ó", "o")
)

df_receptivo = df_receptivo.withColumn(
    "Lugar Ingreso",
    F.regexp_replace("Lugar Ingreso", "ú", "u")
)

# Reemplazar tildes en la columna "Lugar Egreso"
df_receptivo = df_receptivo.withColumn(
    "Lugar Egreso",
    F.regexp_replace("Lugar Egreso", "Á", "A")
)

df_receptivo = df_receptivo.withColumn(
    "Lugar Egreso",
    F.regexp_replace("Lugar Egreso", "É", "E")
)

df_receptivo = df_receptivo.withColumn(
    "Lugar Egreso",
    F.regexp_replace("Lugar Egreso", "Í", "I")
)

df_receptivo = df_receptivo.withColumn(
    "Lugar Egreso",
    F.regexp_replace("Lugar Egreso", "Ó", "O")
)

df_receptivo = df_receptivo.withColumn(
    "Lugar Egreso",
    F.regexp_replace("Lugar Egreso", "Ú", "U")
)

df_receptivo = df_receptivo.withColumn(
    "Lugar Egreso",
    F.regexp_replace("Lugar Egreso", "á", "a")
)

df_receptivo = df_receptivo.withColumn(
    "Lugar Egreso",
    F.regexp_replace("Lugar Egreso", "é", "e")
)

df_receptivo = df_receptivo.withColumn(
    "Lugar Egreso",
    F.regexp_replace("Lugar Egreso", "í", "i")
)

df_receptivo = df_receptivo.withColumn(
    "Lugar Egreso",
    F.regexp_replace("Lugar Egreso", "ó", "o")
)

df_receptivo = df_receptivo.withColumn(
    "Lugar Egreso",
    F.regexp_replace("Lugar Egreso", "ú", "u")
)



In [6]:
# DUDA: Transito, No Corresponde (en departamento, otro Departamento y en Localidad) deberiamos dejarlo como null??
# df_receptivo.select("Departamento").orderBy(F.asc("Departamento")).distinct().show(500,truncate=False)

# DUDA como normalizar (hay ciudades y paises mezclados)
# df_receptivo.groupBy("Residencia").count().show(500,truncate=False)

In [7]:
# Agrupar los valores "Otros", "Sin Datos" y "Desconocido / Sin Datos" en una única categoría "Otros"
df_receptivo = df_receptivo.withColumn(
    "Ocupacion",
    F.when(F.col("Ocupacion").isin("Otros", "Sin Datos", "Desconocido / Sin Datos"), "Otros")
    .otherwise(F.col("Ocupacion"))
)

# Agrupar los valores "Otra Situacion Inactividad", "Desocupado" en una única categoría "Desocupado"
df_receptivo = df_receptivo.withColumn(
    "Ocupacion",
    F.when(F.col("Ocupacion").isin("Otra situacion Inactividad", "Desocupado"), "Desocupado")
    .otherwise(F.col("Ocupacion"))
)


In [8]:
# Actualizar la columna "Estudio" para reemplazar los valores "Sin Datos", "Otros" y null por "Otros"
df_receptivo = df_receptivo.withColumn(
    "Estudio",
    F.when(F.col("Estudio").isin("Sin Datos", "Otros"), "Otros")
    .when(F.col("Estudio").isNull(), "Otros")  # Maneja los valores null
    .otherwise(F.col("Estudio"))
)

In [9]:
# Actualizar la columna "Localidad"
df_receptivo = df_receptivo.withColumn(
    "Localidad",
    F.when(F.col("Localidad").isin("Sin Datos", "Otros"), "Otros")
    .when(F.col("Localidad").isNull(), "Otros")  # Maneja los valores null
    .when(F.col("Localidad").isin("Jaugueriberry"), "Jaureguiberry")
    .otherwise(F.col("Localidad"))
)

In [10]:
# Actualizar la columna "Otra Localidad"
df_receptivo = df_receptivo.withColumn(
    "Otra Localidad",
    F.when(F.col("Otra Localidad").isin("Jaugueriberry"), "Jaureguiberry")
    .otherwise(F.col("Otra Localidad"))
)

In [11]:

# Normalizar columna "Motivo" en df_receptivo
df_receptivo = df_receptivo.withColumn(
    "Motivo",
    F.when(F.col("Motivo") == "Trabajo / Profesional", "Trabajo")
    .when(F.col("Motivo") == "Ocio y vacaciones", "Ocio / Vacaciones")
    .when(F.col("Motivo") == "Visita a familiares y amigos", "Visita familiares / amigos")
    .when(F.col("Motivo") == "Salud / wellness", "Salud")
    .when(F.col("Motivo") == "Religioso", "Religion")
    .otherwise(F.col("Motivo"))  # Mantener el valor original si no hay coincidencia
)

# Mostrar los resultados normalizados
df_receptivo.groupBy("Motivo").count().show(500, truncate=False)

+--------------------------+-----+
|Motivo                    |count|
+--------------------------+-----+
|Trabajo                   |6397 |
|Salud                     |277  |
|Visita familiares / amigos|15282|
|Transito                  |3111 |
|Religion                  |193  |
|Compras                   |249  |
|Estudios                  |292  |
|Ocio / Vacaciones         |36248|
|Segunda Residencia        |2161 |
|Otros                     |1022 |
|MICE                      |101  |
+--------------------------+-----+



In [12]:
# Corregir typo en Appart Hotel
df_receptivo = df_receptivo.withColumn(
    "Alojamiento",
    F.when(F.col("Alojamiento") == "Appart Hotel", "Apart Hotel")
    .when(F.col("Alojamiento") == "Sin Datos", "Otros")
    .otherwise(F.col("Alojamiento"))
)

In [13]:
df_receptivo = df_receptivo.withColumn(
    "TransporteLocal",
    F.when(F.col("TransporteLocal") == "Sin Datos", "Otros")
    .otherwise(F.col("TransporteLocal"))
)

In [14]:
df_receptivo = df_receptivo.withColumn(
    "Lugar Egreso",
    F.when(F.col("Lugar Egreso") == "Sin Datos", "Otros")
    .otherwise(F.col("Lugar Egreso"))
)

In [15]:
df_receptivo = df_receptivo.withColumn(
    "Transporte Internacional de Egreso",
    F.when(F.col("Transporte Internacional de Egreso") == "Sin Datos", "Otros")
    .otherwise(F.col("Transporte Internacional de Egreso"))
)

In [16]:
 # |-- GastoTotal: double (nullable = true)
 # |-- GastoAlojamiento: double (nullable = true)
 # |-- GastoAlimentacion: double (nullable = true)
 # |-- GastoTransporte: double (nullable = true)
 # |-- GastoCultural: double (nullable = true)
 # |-- GastoTours: double (nullable = true)
 # |-- GastoCompras: double (nullable = true)
 # |-- GastoOtros: double (nullable = true)

# Crear una nueva columna que calcule la suma de los otros gastos
df_receptivo = df_receptivo.withColumn(
    "SumaGastos",
    F.col("GastoAlojamiento") + F.col("GastoAlimentacion") + F.col("GastoTransporte") + F.col("GastoCultural") + F.col("GastoTours") + F.col("GastoCompras") + F.col("GastoOtros")
)

# Cuando GastoTotal es mayor que SumaGastos, agregar la diferencia a GastoOtros
df_receptivo = df_receptivo.withColumn(
    "GastoOtros",
    F.when(F.col("GastoTotal") > F.col("SumaGastos"), F.col("GastoOtros") + (F.col("GastoTotal") - F.col("SumaGastos")))
    .otherwise(F.col("GastoOtros"))
)

# Cuando SumaGastos es mayor que GastoTotal, actualizar GastoTotal con el valor de SumaGastos
df_receptivo = df_receptivo.withColumn(
    "GastoTotal",
    F.when(F.col("SumaGastos") > F.col("GastoTotal"), F.col("SumaGastos"))
    .otherwise(F.col("GastoTotal"))
)

df_receptivo = df_receptivo.drop("SumaGastos");

# Emisivo
Contiene los datos de egreso al país

In [17]:
df_emisivo = spark.read.csv('../../obligatorio/lnd/emisivo', header=True, sep=',',inferSchema=True)

Asi podemos visualizar los tipos de datos y columnas

In [18]:
df_emisivo.printSchema()

root
 |-- IdLugarSalida: integer (nullable = true)
 |-- Lugar Salida: string (nullable = true)
 |-- IdTranspSalidad: integer (nullable = true)
 |-- Transporte Internacional de Salida: string (nullable = true)
 |-- FechaSalida: date (nullable = true)
 |-- IdFecSalida: integer (nullable = true)
 |-- FechaEntrada: date (nullable = true)
 |-- IdFecEntrada: integer (nullable = true)
 |-- IdNacionalidad: integer (nullable = true)
 |-- Pais: string (nullable = true)
 |-- IdDeptoResidencia: integer (nullable = true)
 |-- Departamento: string (nullable = true)
 |-- IdMotivo: integer (nullable = true)
 |-- Motivo: string (nullable = true)
 |-- IdOcupacion: integer (nullable = true)
 |-- Ocupacion: string (nullable = true)
 |-- IdNivelEstudio: integer (nullable = true)
 |-- Estudio: string (nullable = true)
 |-- IdDestino: integer (nullable = true)
 |-- Destino: string (nullable = true)
 |-- IdAlojamiento: integer (nullable = true)
 |-- Alojamiento: string (nullable = true)
 |-- IdLugarIngreso: i

### Descripción de las Columnas

- Lugar Salida es el lugar y tiene su propio ID (IdLugarSalida).
- Transporte Internacional de Salida es el medio de transporte internacional utilizado para la salida, con su propio ID (IdTranspSalidad).
- Fecha Salida del viaje, con su propio ID (IdFecSalida).
- Fecha Entrada al país, con su propio ID (IdFecEntrada).
- Nacionalidad junto con el código que representa la nacionalidad del turista (IdNacionalidad).
- País de destino del viaje.
- Departamento Residencia con su propio ID (IdDeptoResidencia).
- Motivo de ingreso al país, con su propio ID (IdMotivo).
- Ocupación (o profesión) del turista, con su propio ID (IdOcupacion).
- Estudio es el nivel de estudio del turista, con su propio ID (IdNivelEstudio).
- Destino dentro del país con su propio ID (IdDestino).
- Alojamiento es el tipo de alojamiento utilizado, con su propio ID (IdAlojamiento).
- Lugar Ingreso es el lugar por donde se ingresó al país, con su propio ID (IdLugarIngreso).
- Transporte Internacional de Ingreso es el medio de transporte internacional utilizado para el ingreso, con su propio ID (IdTranspIngreso).
- Transporte Local es el medio de transporte local utilizado, con su propio ID (IdTranspLocal).
- Estadía es la cantidad de días de estadía.
- Gente es el número de integrantes del grupo encuestado.
- Gasto Total, Gasto Alojamiento, Gasto Alimentación, Gasto Transporte Internacional, Gasto Transporte Local, Gasto Cultural, Gasto Tours, Gasto Compras y Gasto Resto se refieren a los distintos tipos de gastos realizados, medido en dólares americanos corrientes.
- Coef es el Coeficiente Expansión Gasto.
- CoefTot es el Coeficiente Expansión Personas.

In [19]:
# Quitar columnas que no usamos
df_emisivo = df_emisivo.drop("IdLugarSalida", "IdTranspSalidad", "IdFecSalida", "IdFecEntrada", "IdNacionalidad", "IdDeptoResidencia", "IdMotivo", "IdOcupacion", "IdNivelEstudio", "IdDestino", "IdAlojamiento", "IdLugarIngreso", "IdTranspIngreso", "IdTranspLocal");

In [20]:
df_emisivo.printSchema()

root
 |-- Lugar Salida: string (nullable = true)
 |-- Transporte Internacional de Salida: string (nullable = true)
 |-- FechaSalida: date (nullable = true)
 |-- FechaEntrada: date (nullable = true)
 |-- Pais: string (nullable = true)
 |-- Departamento: string (nullable = true)
 |-- Motivo: string (nullable = true)
 |-- Ocupacion: string (nullable = true)
 |-- Estudio: string (nullable = true)
 |-- Destino: string (nullable = true)
 |-- Alojamiento: string (nullable = true)
 |-- Lugar Ingreso: string (nullable = true)
 |-- Transporte Internacional de Ingreso: string (nullable = true)
 |-- Trasporte Local: string (nullable = true)
 |-- Estadia: integer (nullable = true)
 |-- Gente: integer (nullable = true)
 |-- GastoTotal: double (nullable = true)
 |-- GastoAlojamiento: double (nullable = true)
 |-- GastoAlimentacion: double (nullable = true)
 |-- GastoTransporteInternac: double (nullable = true)
 |-- GatoTransporteLocal: double (nullable = true)
 |-- GastoCultural: double (nullable = t

In [21]:
test = df_emisivo.groupBy("Transporte Internacional de Salida").count()


In [22]:
test.show()

# DUDA: Podriamos normalizar Aereo, Terrestre, Marítimo o sería perder datos? 

+----------------------------------+-----+
|Transporte Internacional de Salida|count|
+----------------------------------+-----+
|                             Aereo| 6343|
|                   Terrestre Otros|   60|
|                    Terrestre Auto|11805|
|                Maritimo - Fluvial| 6534|
|                             Otros|   39|
|                     Terrestre Bus| 3283|
+----------------------------------+-----+



In [23]:
# Corregir typo en San Jose
df_emisivo = df_emisivo.withColumn(
    "departamento",
    F.when(F.col("departamento") == "Saqn Jose", "San Jose")
    .otherwise(F.col("departamento"))
)

In [24]:
df_emisivo.groupBy("Motivo").count().show()

# DUDA - MICE podríamos unirlo a Trabajo / Profesión es turismo relacionado a conferencias, exhibiciones, etc.


+--------------------+-----+
|              Motivo|count|
+--------------------+-----+
| Trabajo / Profesion| 3117|
|Visita familiares...| 9117|
|Trabajo remunerad...|   19|
|             Compras| 1036|
|            Estudios|  249|
|           Deportivo|  140|
|Ocio, Recreo, Vac...|13763|
|   Tratamiento Salud|  244|
|Religion y Peregr...|  168|
|               Otros|  201|
|                MICE|   10|
+--------------------+-----+



In [25]:
# Agrupar los valores "Otros", "Sin Datos" y "Desconocido / Sin Datos" en una única categoría "Otros"
df_emisivo = df_emisivo.withColumn(
    "Ocupacion",
    F.when(F.col("Ocupacion").isin("Otros", "Sin Datos", "Desconocido / Sin Datos"), "Otros")
    .otherwise(F.col("Ocupacion"))
)

# Agrupar los valores "Desocupado", "Otra situacion Inactividad" en una única categoría "Desocupado"
df_emisivo = df_emisivo.withColumn(
    "Ocupacion",
    F.when(F.col("Ocupacion").isin("Desocupado", "Otra situacion Inactividad"), "Otros")
    .otherwise(F.col("Ocupacion"))
)

In [26]:
# Actualizar la columna "Estudio" para reemplazar los valores "Sin Datos", "Otros" y null por "Otros"
df_emisivo = df_emisivo.withColumn(
    "Estudio",
    F.when(F.col("Estudio").isin("Sin Datos", "Otros"), "Otros")
    .when(F.col("Estudio").isNull(), "Otros")  # Maneja los valores null
    .otherwise(F.col("Estudio"))
)

In [27]:
# Actualizar la columna "Destino" para reemplazar los valores "Sin Datos" y "Otros" por "Otros"
df_emisivo = df_emisivo.withColumn(
    "Destino",
    F.when(F.col("Destino").isin("Sin Datos", "Otros"), "Otros")
    .otherwise(F.col("Destino"))
)

In [28]:
# Actualizar la columna "Alojamiento" para reemplazar los valores "Sin Datos" y "Otros" por "Otros"
df_emisivo = df_emisivo.withColumn(
    "Alojamiento",
    F.when(F.col("Alojamiento").isin("Sin Datos", "Otros"), "Otros")
    .otherwise(F.col("Alojamiento"))
)

In [29]:
# Corregir typo en Appart Hotel
df_emisivo = df_emisivo.withColumn(
    "Alojamiento",
    F.when(F.col("Alojamiento") == "Appart Hotel", "Apart Hotel")
    .otherwise(F.col("Alojamiento"))
)

In [30]:
# Actualizar la columna "Trasporte Local" para reemplazar los valores "Sin Datos" y "Otros" por "Otros"
df_emisivo = df_emisivo.withColumn(
    "Trasporte Local",
    F.when(F.col("Trasporte Local").isin("Sin Datos", "Otros"), "Otros")
    .otherwise(F.col("Trasporte Local"))
)

In [31]:
df_emisivo = df_emisivo.withColumnRenamed("GastoResto", "GastoOtros")
df_emisivo = df_emisivo.withColumnRenamed("GatoTransporteLocal", "GastoTransporteLocal")

# Asegurarnos de que la suma de todos los gastos y GastoTotal se correspondan

# Crear una nueva columna que calcule la suma de los otros gastos
df_emisivo = df_emisivo.withColumn(
    "SumaGastos",
    F.col("GastoAlojamiento") + F.col("GastoAlimentacion") + F.col("GastoTransporteInternac") + F.col("GastoTransporteLocal") + F.col("GastoCultural") + F.col("GastoTours") + F.col("GastoCompras") + F.col("GastoOtros")
)

# Cuando GastoTotal es mayor que SumaGastos, agregar la diferencia a GastoOtros
df_emisivo = df_emisivo.withColumn(
    "GastoOtros",
    F.when(F.col("GastoTotal") > F.col("SumaGastos"), F.col("GastoOtros") + (F.col("GastoTotal") - F.col("SumaGastos")))
    .otherwise(F.col("GastoOtros"))
)

# Cuando SumaGastos es mayor que GastoTotal, actualizar GastoTotal con el valor de SumaGastos
df_emisivo = df_emisivo.withColumn(
    "GastoTotal",
    F.when(F.col("SumaGastos") > F.col("GastoTotal"), F.col("SumaGastos"))
    .otherwise(F.col("GastoTotal"))
)

df_emisivo = df_emisivo.drop("SumaGastos");


In [32]:
df_emisivo.select("Coef", "CoefTot").show() # DUDA: Que hacemos con estas columnas las podemos borrar porque no entendemos que significan

+------+-------+
|  Coef|CoefTot|
+------+-------+
| 89.61| 448.05|
| 96.18| 480.88|
|129.04| 258.09|
|129.04| 387.13|
| 89.61| 448.05|
| 96.18| 480.88|
|128.81| 386.44|
| 96.18| 384.71|
| 96.18| 384.71|
| 89.61| 179.22|
| 96.18| 288.53|
|128.81| 257.63|
|196.13| 392.26|
|127.77| 255.55|
|196.13| 588.39|
| 96.18| 192.35|
|273.99| 547.99|
|273.99|1095.97|
|273.99| 273.99|
|128.81| 257.63|
+------+-------+
only showing top 20 rows



In [33]:
# Reemplazar tildes en la columna "Lugar Salida"
df_emisivo = df_emisivo.withColumn(
    "Lugar Salida",
    F.regexp_replace("Lugar Salida", "Á", "A")
)

df_emisivo = df_emisivo.withColumn(
    "Lugar Salida",
    F.regexp_replace("Lugar Salida", "É", "E")
)

df_emisivo = df_emisivo.withColumn(
    "Lugar Salida",
    F.regexp_replace("Lugar Salida", "Í", "I")
)

df_emisivo = df_emisivo.withColumn(
    "Lugar Salida",
    F.regexp_replace("Lugar Salida", "Ó", "O")
)

df_emisivo = df_emisivo.withColumn(
    "Lugar Salida",
    F.regexp_replace("Lugar Salida", "Ú", "U")
)

df_emisivo = df_emisivo.withColumn(
    "Lugar Salida",
    F.regexp_replace("Lugar Salida", "á", "a")
)

df_emisivo = df_emisivo.withColumn(
    "Lugar Salida",
    F.regexp_replace("Lugar Salida", "é", "e")
)

df_emisivo = df_emisivo.withColumn(
    "Lugar Salida",
    F.regexp_replace("Lugar Salida", "í", "i")
)

df_emisivo = df_emisivo.withColumn(
    "Lugar Salida",
    F.regexp_replace("Lugar Salida", "ó", "o")
)

df_emisivo = df_emisivo.withColumn(
    "Lugar Salida",
    F.regexp_replace("Lugar Salida", "ú", "u")
)


In [34]:
# Normalizar columna "Motivo" en df_emisivo
df_emisivo = df_emisivo.withColumn(
    "Motivo",
    F.when(F.col("Motivo") == "Trabajo / Profesion", "Trabajo")
    .when(F.col("Motivo") == "Trabajo remunerado Destino", "Trabajo")
    .when(F.col("Motivo") == "Ocio, Recreo, Vacaciones", "Ocio / Vacaciones")
    .when(F.col("Motivo") == "Tratamiento Salud", "Salud")
    .when(F.col("Motivo") == "Religion y Peregrinaciones", "Religion")
    .when(F.col("Motivo") == "Deportivo", "Deporte")
    .otherwise(F.col("Motivo"))  # Mantener el valor original si no hay coincidencia
)

# Cruceros
Contiene los datos de tráfico de cruceros

In [35]:
df_crucero = spark.read.csv('../../obligatorio/lnd/cruceros', header=True, sep=',',inferSchema=True)

Asi podemos visualizar los tipos de datos y columnas

In [36]:
df_crucero.printSchema()

root
 |-- IdCruceros: integer (nullable = true)
 |-- idNacionalidad: integer (nullable = true)
 |-- Pais: string (nullable = true)
 |-- Fecha: date (nullable = true)
 |-- idfecha: integer (nullable = true)
 |-- idPuerto: integer (nullable = true)
 |-- Puerto: string (nullable = true)
 |-- TotalPersonas: integer (nullable = true)
 |-- CantidadVisitas: integer (nullable = true)
 |-- GastoTotal: double (nullable = true)
 |-- GastoTours: double (nullable = true)
 |-- GastoAlimentacion: double (nullable = true)
 |-- GastoTransporte: double (nullable = true)
 |-- GastoShopping: double (nullable = true)
 |-- GastoOtros: double (nullable = true)



### Descripción de las Columnas

- IdCruceros es el código identificatorio de los cruceros.
- Pais corresponde a la nacionalidad del entrevistado y tiene su propio ID (idNacionalidad).
- Fecha indica cuándo se realizó la encuesta y tiene un ID (idfecha).
- Puerto es el puerto de arribo del crucerista con su ID (idPuerto).
- Total Personas indica la cantidad de personas en cada grupo.
- Cantidad Visitas representa el número de visitas anteriores del crucerista.
- Gasto Total, Gasto Tours, Gasto Alimentacion, Gasto Transporte, Gasto Shopping, Gasto Otros corresponde a todos los tipos de gastos realizados y su valor en dólares americanos.

decir que probamos que por ejemplo brasil y brazil no pasaba (por eso no agregamos columna de id a paises y a las demas tablas)

In [37]:
# Cantidad de visitas

In [38]:
df_crucero = df_crucero.fillna({'CantidadVisitas': 0}) # Cambiamos a 0 los que tenian valor null


In [39]:
# Para acordarnos, tenemos que ver si todos los datos que tenemos de dinero estan en la misma moneda

In [40]:
# Quitar columnas que no usamos
df_crucero = df_crucero.drop("idNacionalidad", "idPuerto", "idFecha");


In [41]:
df_crucero.printSchema()

root
 |-- IdCruceros: integer (nullable = true)
 |-- Pais: string (nullable = true)
 |-- Fecha: date (nullable = true)
 |-- Puerto: string (nullable = true)
 |-- TotalPersonas: integer (nullable = true)
 |-- CantidadVisitas: integer (nullable = false)
 |-- GastoTotal: double (nullable = true)
 |-- GastoTours: double (nullable = true)
 |-- GastoAlimentacion: double (nullable = true)
 |-- GastoTransporte: double (nullable = true)
 |-- GastoShopping: double (nullable = true)
 |-- GastoOtros: double (nullable = true)



In [42]:
# Asegurarnos de que la suma de todos los gastos y GastoTotal se correspondan

# Crear una nueva columna que calcule la suma de los otros gastos
df_crucero = df_crucero.withColumn(
    "SumaGastos",
    F.col("GastoTours") + F.col("GastoAlimentacion") + F.col("GastoTransporte") + F.col("GastoShopping") + F.col("GastoOtros")
)

# Cuando GastoTotal es mayor que SumaGastos, agregar la diferencia a GastoOtros
df_crucero = df_crucero.withColumn(
    "GastoOtros",
    F.when(F.col("GastoTotal") > F.col("SumaGastos"), F.col("GastoOtros") + (F.col("GastoTotal") - F.col("SumaGastos")))
    .otherwise(F.col("GastoOtros"))
)

# Cuando SumaGastos es mayor que GastoTotal, actualizar GastoTotal con el valor de SumaGastos
df_crucero = df_crucero.withColumn(
    "GastoTotal",
    F.when(F.col("SumaGastos") > F.col("GastoTotal"), F.col("SumaGastos"))
    .otherwise(F.col("GastoTotal"))
)

df_crucero = df_crucero.drop("SumaGastos");


In [43]:
# Para no perder datos, los datos de Crucero con Pais = "Sin Datos" no los borramos pero no vamos a poder considerarlos cuando hagamos mapas de calor por ejemplo. (solo es 1 registro)

# Usando filter
df_sin_datos = df_crucero.filter(F.col("Pais") == "Sin Datos")

print(df_sin_datos.count())

1


# Paises
Contiene los mappeos de paises a el nombre en inglés y su continente

In [57]:
df_paises = spark.read.csv('../../obligatorio/lnd/paises', header=True, sep=',',inferSchema=True)

## Guardar los data frames en rfn

In [58]:
df_crucero.write.csv('/obligatorio/rfn/cruceros', header=True, mode='overwrite')
df_emisivo.write.csv('/obligatorio/rfn/emisivo', header=True, mode='overwrite')
df_receptivo.write.csv('/obligatorio/rfn/receptivo', header=True, mode='overwrite')
df_paises.write.csv('/obligatorio/rfn/paises', header=True, mode='overwrite')


## Mover los archivos de lnd a raw

In [48]:
hadoop_conf = spark._jsc.hadoopConfiguration()

In [51]:
fs = spark._jvm.org.apache.hadoop.fs.FileSystem.get(hadoop_conf)

In [53]:
ruta_origen_receptivo = "/obligatorio/lnd/receptivo"
ruta_destino_receptivo = "/obligatorio/raw/receptivo"

# Mover el archivo de lnd a raw
fs.rename(
    spark._jvm.org.apache.hadoop.fs.Path(ruta_origen_receptivo),
    spark._jvm.org.apache.hadoop.fs.Path(ruta_destino_receptivo)
)

True

In [59]:
ruta_origen_emisivo = "/obligatorio/lnd/emisivo"
ruta_destino_emisivo = "/obligatorio/raw/emisivo"

# Mover el archivo de lnd a raw
fs.rename(
    spark._jvm.org.apache.hadoop.fs.Path(ruta_origen_emisivo),
    spark._jvm.org.apache.hadoop.fs.Path(ruta_destino_emisivo)
)

True

In [60]:
ruta_origen_cruceros = "/obligatorio/lnd/cruceros"
ruta_destino_cruceros = "/obligatorio/raw/cruceros"

# Mover el archivo de lnd a raw
fs.rename(
    spark._jvm.org.apache.hadoop.fs.Path(ruta_origen_cruceros),
    spark._jvm.org.apache.hadoop.fs.Path(ruta_destino_cruceros)
)

True

In [61]:
ruta_origen_paises = "/obligatorio/lnd/paises"
ruta_destino_paises = "/obligatorio/raw/paises"

# Mover el archivo de lnd a raw
fs.rename(
    spark._jvm.org.apache.hadoop.fs.Path(ruta_origen_paises),
    spark._jvm.org.apache.hadoop.fs.Path(ruta_destino_paises)
)

True